In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import os

In [3]:
# 本教程中使用的文本文件已经进行过一些典型的预处理，主要包括删除了文档页眉和页脚，行号，章节标题。请下载这些已经被局部改动过的文件。
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    text_dir = keras.utils.get_file(name, origin=DIRECTORY_URL+name)
# Downloads a file from a URL if it not already in the cache.  
parent_dir = os.path.dirname(text_dir)
print(parent_dir)

/Users/mac/.keras/datasets


## 将文本加载到数据集中
#### 迭代整个文件，将整个文件加载到自己的数据集中。

#### 每个样本都需要单独标记，所以请使用 tf.data.Dataset.map 来为每个样本设定标签。这将迭代数据集中的每一个样本并且返回（ example, label ）对。

In [83]:
def labeler(example, index):
    return example, tf.cast(index, tf.int64) # tf.cast()函数的作用是执行 tensorflow 中张量数据类型转换，即将index转换为int64

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
    print(i,file_name)
    lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
    print(lines_dataset)
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    print(labeled_dataset)
    labeled_data_sets.append(labeled_dataset)
print(type(labeled_data_sets[0]))

0 cowper.txt
<TextLineDatasetV2 shapes: (), types: tf.string>
<MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>
1 derby.txt
<TextLineDatasetV2 shapes: (), types: tf.string>
<MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>
2 butler.txt
<TextLineDatasetV2 shapes: (), types: tf.string>
<MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [7]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
    all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [93]:
for ex,i in all_labeled_data.take(20):
    print(ex)

tf.Tensor(b"Upon his lofty vessel's prow, and watch'd", shape=(), dtype=string)
tf.Tensor(b'To burst the phalanx, and confusion sent', shape=(), dtype=string)
tf.Tensor(b'Check his advance, such vigour Pallas gave;', shape=(), dtype=string)
tf.Tensor(b'her delicate hand with the gold pin of the woman\'s brooch."', shape=(), dtype=string)
tf.Tensor(b'And seek it instant. It were much unmeet', shape=(), dtype=string)
tf.Tensor(b'Myrsinus and the Hyrminian plain between,', shape=(), dtype=string)
tf.Tensor(b'greatest king, and had most men under him.', shape=(), dtype=string)
tf.Tensor(b"'Gan move toward them, and the Greeks again", shape=(), dtype=string)
tf.Tensor(b'Meantime arrived, to whose approach the wives', shape=(), dtype=string)
tf.Tensor(b'To Argos, ere events shall yet have proved', shape=(), dtype=string)
tf.Tensor(b'consulted the scales of destiny, he directs his lightning against the', shape=(), dtype=string)
tf.Tensor(b'High-mettled horses, well survey and search', shape=(

## 将文本编码成数字
#### 机器学习基于的是数字而非文本，所以字符串需要被转化成数字列表。 为了达到此目的，我们需要构建文本与整数的一一映射

In [ ]:
# 建立词汇表

# 首先，通过将文本标记为单独的单词集合来构建词汇表。在 TensorFlow 和 Python 中均有很多方法来达成这一目的。在本教程中:

# 迭代每个样本的 numpy 值。
# 使用 tfds.features.text.Tokenizer 来将其分割成 token。
# 将这些 token 放入一个 Python 集合中，借此来清除重复项。
# 获取该词汇表的大小以便于以后使用。

In [97]:
tokenizer = tfds.features.text.Tokenizer()#Tokenizer将句子转化为单词列表
vocabulary_set = set()

for text_tensor, _ in all_labeled_data:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())# 将句子转化为单词列表
    vocabulary_set.update(some_tokens)# 放入set中去重

vocab_size = len(vocabulary_set)
vocab_size

17178

## 样本编码

#### 通过传递 vocabulary_set 到 tfds.features.text.TokenTextEncoder 来构建一个编码器。编码器的 encode 方法传入一行文本，返回一个整数列表。

In [98]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)# 传入set构建编码器
type(encoder)

tensorflow_datasets.core.features.text.text_encoder.TokenTextEncoder

In [99]:
# 编码器使用示例
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)
encoded_example = encoder.encode(example_text)
print(encoded_example)

b"Upon his lofty vessel's prow, and watch'd"
[153, 8145, 10785, 11709, 13276, 13075, 5471, 4253, 16088]


### 现在，在数据集上运行编码器（通过将编码器打包到 tf.py_function 并且传参至数据集的 map 方法的方式来运行）。

In [100]:
def encode(text_tensor, label):
    '''使用前面构建的编码器对数据进行编码'''
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
    encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
    encoded_text.set_shape([None])# 设置数据集的形状
    label.set_shape([])# 设置label的形状

    return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)# 数据集的 map 方法的方式来运行

## 将数据集分割为测试集和训练集且进行分支
#### 使用 tf.data.Dataset.take 和 tf.data.Dataset.skip 来建立一个小一些的测试数据集和稍大一些的训练数据集。
#### 在数据集被传入模型之前，数据集需要被分批。最典型的是，每个分支中的样本大小与格式需要一致。但是数据集中样本并不全是相同大小的（每行文本字数并不相同）。因此，使用 tf.data.Dataset.padded_batch（而不是 batch ）将样本填充到相同的大小。


In [104]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes = ([None],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE,padded_shapes = ([None],[]))

In [105]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]


In [108]:
# 由于我们引入了一个新的 token 来编码（填充零），因此词汇表大小增加了一个。

vocab_size += 1

In [106]:
model = tf.keras.Sequential()
# 第一层将整数表示转换为密集矢量嵌入
model.add(tf.keras.layers.Embedding(vocab_size, 64))
# 下一层是 LSTM 层，它允许模型利用上下文中理解单词含义。
# LSTM 上的双向包装器有助于模型理解当前数据点与其之前和之后的数据点的关系。
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
# 一个或多个紧密连接的层
# 编辑 `for` 行的列表去检测层的大小
for units in [64, 64]:
    model.add(tf.keras.layers.Dense(units, activation='relu'))

# 输出层。第一个参数是标签个数。
model.add(tf.keras.layers.Dense(3, activation='softmax'))


# 对于一个 softmax 分类模型来说，通常使用 sparse_categorical_crossentropy 作为其损失函数。
# 你可以尝试其他的优化器，但是 adam 是最常用的。
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          1099456   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 195       
Total params: 1,178,115
Trainable params: 1,178,115
Non-trainable params: 0
_________________________________________________________________


In [107]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 145s 209ms/step - loss: 0.5105 - accuracy: 0.7534 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
697/697 [==============================] - 130s 187ms/step - loss: 0.2961 - accuracy: 0.8696 - val_loss: 0.3697 - val_accuracy: 0.8356
Epoch 3/3
697/697 [==============================] - 137s 197ms/step - loss: 0.2263 - accuracy: 0.9004 - val_loss: 0.3929 - val_accuracy: 0.8396


In [109]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))

79/79 [==============================] - 11s 140ms/step - loss: 0.3929 - accuracy: 0.8396

Eval loss: 0.39293034789682946, Eval accuracy: 0.8396000266075134
